# Linear Regression: Predicting Taxi-Out Delay
## Given the dataset, predict the runway time of the flight.

#### Task Details
1. Go throught the dataset and perform preprocessing and then perform a 90:10 split and for train and test pruposes.
2. Firstly label encode the columns which are required.
3. Your target or **y variable is TAXI-OUT time**. Use all 8 algorithms above on the dataset with loss score as RMSE (Root mean Square Error).
4. Now, One-Hot encode all the data points and preform the 3rd Step again.

Keep in mind that you will be using the same splitted dataframe for all the training and testing and should not split again.

##### Some models to consider:-

Linear Models:-

1. Linear Regression
2. Ridge Regression(Popularily L1)
3. Lasso Regression(Popularily L2)

Non linear Models:-

1. KNN model
2. SVR
3. Naive Bayes
4. Random Forest
5. LightGBM(Tree Based Model)

##### Expected Submission
Open Submission till end of July.
Submit a Notebook: End-Goal: We will see how label encoding or one hot encoding is better for the model and which out of the 8 algorithms which is the best. Just a cojparative study and plots to understand the results on bigger datasets.

##### Evaluation
Having a complete report along with comparative graphs.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

### Configuring the Data Frame
> **Source:** MUHAMMET ALI BÜYÜKNACAR'S [work](https://www.kaggle.com/buyuknacar/jfk-flight-notebook). 

In [2]:
pd.set_option('display.width', 1200)
pd.set_option('display.max_columns', 25)

### Loading the data from csv file

In [3]:
taxi = pd.read_csv('M1_final.csv')
taxi.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,DEST,DEP_DELAY,CRS_ELAPSED_TIME,DISTANCE,CRS_DEP_M,DEP_TIME_M,CRS_ARR_M,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,sch_dep,sch_arr,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,323,448,48,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,333,531,48,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,341,482,48,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,343,513,48,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,356,499,46,32,58,W,24,35,29.91,Fair / Windy,9,17,13


### Editing the column names for clarity
> **Source:** MUHAMMET ALI BÜYÜKNACAR'S [work](https://www.kaggle.com/buyuknacar/jfk-flight-notebook). 

In [4]:
col_names = {"OP_UNIQUE_CARRIER":"CARRIER_CODE",
                "TAIL_NUM":"FLIGHT_NO",
                "CRS_ELAPSED_TIME":"SCHEDULED_DURATION",
                "CRS_DEP_M":"SCHEDULED_DEPARTURE",
                "DEP_TIME_M":"ACTUAL_DEP_TIME",
                "CRS_ARR_M":"SCHEDULED_ARRIVAL",
                "sch_dep":"FLT_SCH_ARRIVAL",
                "sch_arr":"FLT_SCH_DEPARTURE"}

taxi = taxi.rename(col_names, axis=1)
taxi.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CARRIER_CODE,FLIGHT_NO,DEST,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
0,11,1,5,B6,N828JB,CHS,-1,124,636,324,323,448,48,34,58,W,25,38,29.86,Fair / Windy,9,17,14
1,11,1,5,B6,N992JB,LAX,-7,371,2475,340,333,531,48,34,58,W,25,38,29.86,Fair / Windy,9,17,15
2,11,1,5,B6,N959JB,FLL,40,181,1069,301,341,482,48,34,58,W,25,38,29.86,Fair / Windy,9,17,22
3,11,1,5,B6,N999JQ,MCO,-2,168,944,345,343,513,48,34,58,W,25,38,29.86,Fair / Windy,9,17,12
4,11,1,5,DL,N880DN,ATL,-4,139,760,360,356,499,46,32,58,W,24,35,29.91,Fair / Windy,9,17,13


### Preparing the Data for Analysis

#### Aggregating using: min, max, means, sum, distinct and count.

In [20]:
pd.set_option('display.max_columns', 120) # configure dataframe to show all columns

dest = taxi.drop(columns=['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']) # create new dataframe from taxi and drop the date columns

dest.groupby(['DEST']).describe() # creates dataframe grouped by destinations, showing details on relevant columns' numerical data.  

DEP_DELAY                                                      SCHEDULED_DURATION                                                          DISTANCE                                                      SCHEDULED_DEPARTURE                                                                  ACTUAL_DEP_TIME                                                                    SCHEDULED_ARRIVAL                                                                  Temperature                                                     Humidity                                                       Wind Speed                                                    Wind Gust                                                Pressure                                                              FLT_SCH_ARRIVAL                                                      FLT_SCH_DEPARTURE                                                     TAXI_OUT                                                   
         count       mean        std   min   25%  50%    75%    max              count        mean       std    min    25%    50%    75%    max    count    mean  std     min     25%     50%     75%     max               count         mean         std     min     25%     50%     75%     max           count         mean         std     min      25%     50%      75%     max             count         mean         std     min     25%     50%     75%     max       count       mean       std   min    25%   50%   75%   max    count       mean        std   min    25%   50%    75%   max      count       mean       std  min   25%   50%    75%   max     count      mean        std  min  25%  50%  75%   max    count       mean       std    min      25%     50%      75%    max           count       mean        std   min   25%   50%    75%   max             count       mean       std   min   25%   50%    75%   max    count       mean       std   min   25%   50%   75%   max
DEST                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
ABQ       58.0  10.327586  39.259098 -13.0 -6.75 -3.0   7.00  215.0               58.0  310.258621  3.522162  307.0  307.0  307.0  314.0  314.0     58.0  1826.0  0.0  1826.0  1826.0  1826.0  1826.0  1826.0                58.0  1195.844828    4.697150  1189.0  1193.0  1200.0  1200.0  1200.0            58.0  1206.172414   40.006200  1178.0  1190.00  1194.0  1203.25  1415.0              58.0  1386.103448    1.682622  1383.0  1387.0  1387.0  1387.0  1387.0        58.0  41.758621  7.472312  26.0  36.00  42.0  48.0  56.0     58.0  62.327586  23.869814  10.0  49.25  61.0  84.25  97.0       58.0  11.948276  5.325946  0.0  7.25  12.0  16.00  25.0      58.0  5.551724  11.050399  0.0  0.0  0.0  0.0  33.0     58.0  30.083966  0.305424  29.38  29.8925  30.095  30.2975  30.73            58.0  37.189655   6.974784  19.0  33.0  40.0  41.75  47.0              58.0  31.465517  5.737582  13.0  28.0  31.0  33.75  43.0     58.0  22.620690  8.243717  11.0  15.0  21.0  29.0  41.0
ATL      795.0   6.725786  26.290094 -13.0 -4.00 -1.0   6.50  341.0              795.0  158.148428  6.547875  136.0  

In [6]:
taxi.groupby(['DEST']).count()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,CARRIER_CODE,FLIGHT_NO,DEP_DELAY,SCHEDULED_DURATION,DISTANCE,SCHEDULED_DEPARTURE,ACTUAL_DEP_TIME,SCHEDULED_ARRIVAL,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Condition,FLT_SCH_ARRIVAL,FLT_SCH_DEPARTURE,TAXI_OUT
DEST,,,,,,,,,,,,,,,,,,,,,,
ABQ,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58
ATL,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795,795
AUS,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485,485
BNA,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366,366
BOS,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243,1243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SMF,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82
SRQ,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105
STT,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62


### Splitting the dataset (90:10)
> See Real Python's [guide](https://realpython.com/train-test-split-python-data/) for splitting data with scikit-learn and python.
 
> See [this](https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas) stack overflow thread for examples using pandas data frames. 

In [ ]:
#sklearn.model_selection.train_test_split(taxi)
train, test = train_test_split(taxi, test_size=0.1) #test_size set to 0.1 because we want 0.9 train_size